### Tool calling

This is just a prototyping notebook for tool calling with langchain and ollama

In [1]:
#from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_ollama import ChatOllama, OllamaLLM
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import logging
from langchain.chat_models import init_chat_model

from tool_functions import *

In [2]:
model = init_chat_model(
    "llama3.2",
    model_provider="ollama",
    temperature=0.5,
    max_tokens=100,
)

In [3]:
#model = ChatOllama(model="llama3.2", format="json", temperature=0)
model_with_tools = model.bind_tools(tools=get_tools())

In [4]:
messages = [HumanMessage(content="What is the current electricity price in Finland?")]
response = model_with_tools.invoke(messages)
print(response.tool_calls)

[{'name': 'fetch_current_electricity_price', 'args': {}, 'id': 'c2241cf3-a86a-4a5c-9935-d0d811b3f1ea', 'type': 'tool_call'}]


In [5]:
response.tool_calls

[{'name': 'fetch_current_electricity_price',
  'args': {},
  'id': 'c2241cf3-a86a-4a5c-9935-d0d811b3f1ea',
  'type': 'tool_call'}]

In [6]:
messages = [
    SystemMessage(content="""You are a helpful assistant and you can call functions to get information.
    You can call multiple functions if necessary.
                  """
                  ),
    HumanMessage(content="What is the current date? Also what is current time?"),
]

In [7]:
ai_message = model_with_tools.invoke(messages)
messages.append(ai_message)
for tool_call in ai_message.tool_calls:
    selected_tool = tool_map[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[SystemMessage(content='You are a helpful assistant and you can call functions to get information.\n    You can call multiple functions if necessary.\n                  ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the current date? Also what is current time?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-27T19:42:01.314577523Z', 'done': True, 'done_reason': 'stop', 'total_duration': 909250109, 'load_duration': 17007963, 'prompt_eval_count': 895, 'prompt_eval_duration': 140000000, 'eval_count': 28, 'eval_duration': 316000000, 'model_name': 'llama3.2'}, id='run-3020a388-52c5-4362-be8f-3be88dce5598-0', tool_calls=[{'name': 'get_current_date', 'args': {}, 'id': '000ad38d-4401-4950-ad42-1cb0989113b8', 'type': 'tool_call'}, {'name': 'get_current_time', 'args': {}, 'id': '63a4fc11-78e5-4c55-9ba6-1b907c3d4d90', 'type': 'tool_call'}], usage_metadata={'inp

In [8]:
response = model_with_tools.invoke(messages)
print(response)

content='The current date is April 27, 2025. The current time is 10:42 PM.' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-04-27T19:42:05.661919914Z', 'done': True, 'done_reason': 'stop', 'total_duration': 212615509, 'load_duration': 15976174, 'prompt_eval_count': 140, 'prompt_eval_duration': 4000000, 'eval_count': 23, 'eval_duration': 191000000, 'model_name': 'llama3.2'} id='run-6810b470-cf5c-4aab-985c-92cc05e4c1f7-0' usage_metadata={'input_tokens': 140, 'output_tokens': 23, 'total_tokens': 163}


### Using prompt templates

In [55]:
# male a promt template
prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant and you can call functions to get information."),
        ("human", "{user_input}"),        
    ]
)

In [157]:
#res=prompt.invoke({"user_input": "What is the current date? Also what is current time?"})
#res.messages

In [158]:
#res=(prompt | model_with_tools).invoke({"user_input": "What is the current date? Also what is current time?"})
# (prompt | model_with_tools).invoke({"user_input": "What is the current date? Also what is current time?"})

In [56]:

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Set up the function to call the model
def call_model(input_text, prompt):
    '''
    Calls the AI model with the provided input text and returns the response.

    Args:
        prompt (ChatPromptTemplate): The prompt template to use for the model.
        input_text (str): The input text to send to the AI model.

    Returns:
        str: The response from the AI model.
        prompt (ChatPromptTemplate): The updated prompt with the model response.
    '''
    chain = prompt | model_with_tools
    try:
        response = chain.invoke({"user_input": input_text})
        logger.info(f"Model response: {response}")
        # Check if the response contains tool calls
        if response.tool_calls:
            for tool_call in response.tool_calls:
                selected_tool = tool_map[tool_call["name"].lower()]
                tool_msg = selected_tool.invoke(tool_call)
                logger.info(f"Tool response: {tool_msg}")
                # Append the tool response to the messages
                prompt.messages.append(response)
                prompt.messages.append(tool_msg)
                # call the model again with the updated messages
            return call_model(input_text, prompt)
        else:
            logger.info("No tool calls in the response.")
            # update the prompt with the model response
            prompt.messages.append(response)
            new_prompt = ChatPromptTemplate.from_messages(
                prompt.messages + [
                    ("human", "{user_input}"),
                ]
            )
        return response, new_prompt
    except Exception as e:
        logger.error(f"Error calling model: {e}")
        return None, prompt

In [57]:
input_text = "What is the current date? Also what is the current time?"
response, prompt = call_model(input_text, prompt=prompt)
if response:
    print(f"Response: {response}")
else:
    print("Failed to get a response from the model.")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:__main__:Model response: content='' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-04-27T19:58:25.727514019Z', 'done': True, 'done_reason': 'stop', 'total_duration': 476496517, 'load_duration': 13563681, 'prompt_eval_count': 885, 'prompt_eval_duration': 151000000, 'eval_count': 28, 'eval_duration': 309000000, 'model_name': 'llama3.2'} id='run-35c7cd7d-c511-410c-bf75-0bcd32200f4f-0' tool_calls=[{'name': 'get_current_date', 'args': {}, 'id': 'ebf3a944-a37e-40ed-988c-bdb0ceefe4bf', 'type': 'tool_call'}, {'name': 'get_current_time', 'args': {}, 'id': 'be15cef8-ab82-4e0a-9ca3-a68ccd3af7a3', 'type': 'tool_call'}] usage_metadata={'input_tokens': 885, 'output_tokens': 28, 'total_tokens': 913}
INFO:__main__:Tool response: content='2025-04-27' name='get_current_date' tool_call_id='ebf3a944-a37e-40ed-988c-bdb0ceefe4bf'
INFO:__main__:Tool response: content='22:58' name='get_curren

Response: content='The current date is April 27, 2025.\nThe current time is 10:58 PM.' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-04-27T19:58:25.948185786Z', 'done': True, 'done_reason': 'stop', 'total_duration': 216967819, 'load_duration': 15902874, 'prompt_eval_count': 166, 'prompt_eval_duration': 5000000, 'eval_count': 23, 'eval_duration': 194000000, 'model_name': 'llama3.2'} id='run-4fbc695c-1845-49ee-a786-92159dab3748-0' usage_metadata={'input_tokens': 166, 'output_tokens': 23, 'total_tokens': 189}


In [58]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant and you can call functions to get information.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-27T19:58:25.727514019Z', 'done': True, 'done_reason': 'stop', 'total_duration': 476496517, 'load_duration': 13563681, 'prompt_eval_count': 885, 'prompt_eval_duration': 151000000, 'eval_count': 28, 'eval_duration': 309000000, 'model_name': 'llama3.2'}, id='run-35c7cd7d-c511-410c-bf75-0bcd32200f4f-0', tool_calls=[{'name': 'get_current_date', 'args': {}, 'id': 'ebf3a944-a37e-40ed-988c-bdb0ceefe4bf', 'type': 'tool_call'}, {'name': 'get_current_time', 'args': {}, 'id': 'be15cef8-ab82-4e0a-9ca3-a68ccd3af

In [60]:
response, prompt = call_model("what is 12.34 - 337.1 using a tool?", prompt)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:__main__:Model response: content='' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-04-27T19:59:08.095014072Z', 'done': True, 'done_reason': 'stop', 'total_duration': 476966826, 'load_duration': 15581881, 'prompt_eval_count': 1084, 'prompt_eval_duration': 152000000, 'eval_count': 22, 'eval_duration': 298000000, 'model_name': 'llama3.2'} id='run-339198b2-4b4e-4f5d-810f-6746469ad80a-0' tool_calls=[{'name': 'subtract', 'args': {'a': '12', 'b': '337'}, 'id': '66fb2750-d489-4aa5-9cb7-e976d0a7978f', 'type': 'tool_call'}] usage_metadata={'input_tokens': 1084, 'output_tokens': 22, 'total_tokens': 1106}
INFO:__main__:Tool response: content='-325' name='subtract' tool_call_id='66fb2750-d489-4aa5-9cb7-e976d0a7978f'
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:__main__:Model response: content='The result of 12.34 - 337.1 is -324.66 (rounded t

In [61]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant and you can call functions to get information.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-27T19:58:25.727514019Z', 'done': True, 'done_reason': 'stop', 'total_duration': 476496517, 'load_duration': 13563681, 'prompt_eval_count': 885, 'prompt_eval_duration': 151000000, 'eval_count': 28, 'eval_duration': 309000000, 'model_name': 'llama3.2'}, id='run-35c7cd7d-c511-410c-bf75-0bcd32200f4f-0', tool_calls=[{'name': 'get_current_date', 'args': {}, 'id': 'ebf3a944-a37e-40ed-988c-bdb0ceefe4bf', 'type': 'tool_call'}, {'name': 'get_current_time', 'args': {}, 'id': 'be15cef8-ab82-4e0a-9ca3-a68ccd3af

In [62]:
response.tool_calls

[]

In [64]:
response.content

'The result of 12.34 - 337.1 is -324.66 (rounded to two decimal places).'